In [2]:
import os
from openai import OpenAI

api_key = os.getenv("DEEPSEEK_API_KEY");
if not api_key:
    raise ValueError("No Environment Value DEEPSEEK_API_KEY");
client = OpenAI(
    api_key = api_key,
    base_url = "https://api.deepseek.com/v1",
)

In [3]:
#定义一个embedding模型，使用milvus_model来生成文本
import os

os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

from pymilvus import model as milvus_model

embedding_model = milvus_model.DefaultEmbeddingFunction()

/home/hopewang123456/conda3/envs/deepseek/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
from pymilvus import MilvusClient
from tqdm import tqdm
from glob import glob
def query_constellation(question: str) -> str:
    text_lines = []
    for file_path in glob("constellation.md", recursive=True):
        with open(file_path, "r") as file:
            file_text = file.read()
        text_lines += file_text.split("# ")
    milvus_client = MilvusClient(uri='./milvus_demo.db')
    collection_name = "my_constellation_rag_collection"
    if milvus_client.has_collection(collection_name):
        milvus_client.drop_collection(collection_name)
    test_embedding = embedding_model.encode_queries(["This is a test"])[0]
    embedding_dim = len(test_embedding)
    milvus_client.create_collection(
        collection_name=collection_name,
        dimension=embedding_dim,
        metric_type="IP",
        consistency_level="Strong",
    )
    data = []
    doc_embeddings = embedding_model.encode_documents(text_lines)
    for i, line in enumerate(tqdm(text_lines, desc="Creating embeddings")):
        data.append({"id": i, "vector": doc_embeddings[i], "text": line})
    milvus_client.insert(collection_name=collection_name, data=data)
    search_res = milvus_client.search(
        collection_name=collection_name,
        data=embedding_model.encode_queries(
            [question]
        ),
        limit=10,
        search_params={"metric_type": "IP", "params": {}},
        output_fields=["text"]
    )
    return search_res

In [21]:
SYSTEM_PROMPT = """
你是一个资深的小红书爆款文案专家，擅长结合最新星座趋势和人性特点，创作引人入胜、高互动、高转化的笔记文案。

你的任务是根据用户提供的和星座相关的问题，生成包含标题、正文、相关标签和表情符号的完整小红书笔记。

请始终采用'Thought-Action-Observation'模式进行推理和行动。文案风格需活泼、幽默、真诚、富有感染力。当完成任务后，请以JSON格式直接输出最终文案，格式如下：
```json
{
   "title":"小红书标题",
   "body":"小红书正文",
   "hashtags":["#标签1","#标签2","#标签3","#标签4","#标签5"],
   "emojis": ["✨", "🔥", "💖"]
}
```
在生成文案前，请务必先思考并收集足够的信息
"""

In [22]:
TOOLS_DEFINITION = [
    {
        "type": "function",
        "function": {
            "name": "query_product_database",
            "description": "查询内部产品数据库，获取不同星座已经有的相关信息。",
            "parameters": {
                "type": "object",
                "properties": {
                    "product_name": {
                        "type": "string",
                        "description": "要查询的星座名称，例如'白羊座'"
                    }
                },
                "required": ["product_name"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "generate_emoji",
            "description": "根据提供的文本内容，生成一组适合小红书风格的表情符号。",
            "parameters": {
                "type": "object",
                "properties": {
                    "context": {
                        "type": "string",
                        "description": "文案的关键内容或情感，例如'惊喜效果'、'补水保湿'"
                    }
                },
                "required": ["context"]
            }
        }
    }
]

In [23]:
import random # 用于模拟生成表情
import time # 用于模拟网络延迟
import json

def query_constellation_database(product_name: str) -> str:
    """模拟查询向量数据库，返回预设的产品信息。"""
    search_res = query_constellation(product_name)
    retrieved_lines_with_distances = [
        (res["entity"]["text"], res["distance"]) for res in search_res[0]
    ]
    print(json.dumps(retrieved_lines_with_distances, indent=4))
    context = "\n".join([line_with_distance[0] for line_with_distance in retrieved_lines_with_distances])
    return context
def mock_generate_emoji(context: str) -> list:
    """模拟生成表情符号，根据上下文提供常用表情。"""
    print(f"[Tool Call] 模拟生成表情符号，上下文：{context}")
    time.sleep(0.2) # 模拟生成延迟
    if "水象" in context or "双鱼" in context or "天蝎" or "巨蟹" in context in context:
        return ["💦", "💧", "🌊", "✨"]
    elif "火象" in context or "白羊" in context or "狮子" in context or "射手" in context:
        return ["🔥", "⚡", "🎯", "🚀"]
    elif "土象" in context or "摩羯" in context or "金牛" in context or "处女" in context:
        return ["🌿", "⛰️", "💎", "🪨"]
    elif "风象" in context or "天秤" in context or "水瓶" in context or "双子" in context:
        return ["💨", "🌀", "📡", "💭"]
    elif "熬夜" in context or "疲惫" in context:
        return ["😭", "😮‍💨", "😴", "💡"]
    elif "好物" in context or "推荐" in context:
        return ["✅", "👍", "⭐", "🛍️"]
    else:
        return random.sample(["✨", "🔥", "💖", "💯", "🎉", "👍", "🤩", "💧", "🌿"], k=min(5, len(context.split())))

# 将模拟工具函数映射到一个字典，方便通过名称调用
available_tools = {
    "query_product_database": query_constellation_database,
    "generate_emoji": mock_generate_emoji,
}

In [24]:
import json
import re

def generate_rednote(product_name: str, tone_style: str = "活泼甜美", max_iterations: int = 5) -> str:
    """
    使用 DeepSeek Agent 生成小红书爆款文案。
    
    Args:
        product_name (str): 要生成文案的产品名称。
        tone_style (str): 文案的语气和风格，如"活泼甜美"、"知性"、"搞怪"、"幽默"等。
        max_iterations (int): Agent 最大迭代次数，防止无限循环。
        
    Returns:
        str: 生成的爆款文案（JSON 格式字符串）。
    """
    print(f"\n🚀 启动小红书文案生成助手，产品：{product_name}，风格：{tone_style}\n")

    messages = [
        {
            "role": "system", "content": SYSTEM_PROMPT
        },{
            "role": "user", "content": f"请为产品「{product_name}」生成一篇小红书爆款文案。要求：语气{tone_style}，包含标题、正文、至少5个相关标签和5个表情符号。请以完整的JSON格式输出，并确保JSON内容用markdown代码块包裹（例如：```json{{...}}```）。"
        }
    ]

    iteration_count = 0
    final_response = None

    while iteration_count < max_iterations:
        iteration_count += 1
        print(f"-- Iteration {iteration_count} --")

        try:
            response = client.chat.completions.create(
                model = "deepseek-chat",
                messages = messages,
                tools = TOOLS_DEFINITION,
                tool_choice="auto"
            )

            response_message = response.choices[0].message

            if response_message.tool_calls:
                print("Agent: 决定调用工具....")
                messages.append(response_message)

                tool_outputs = []
                for tool_call in response_message.tool_calls:
                    function_name = tool_call.function.name
                    function_args = json.loads(tool_call.function.arguments) if tool_call.function.arguments else {}

                    print(f"Agent Action: 调用工具 '{function_name}', 参数：{function_args}")

                    if function_name in available_tools:
                        tool_function = available_tools[function_name]
                        tool_result = tool_function(**function_args)
                        print(f"Observation: 工具返回结果： {tool_result}")
                        tool_outputs.append({
                            "tool_call_id": tool_call.id,
                            "role": "tool",
                            "content": str(tool_result)
                        })
                    else:
                        error_message = f"错误：未知的工具 '{function_name}'"
                        print(error_message)
                        tool_outputs.append({
                            "tool_call_id": tool_call.id,
                            "role": "tool",
                            "content": error_message
                        })
                messages.extend(tool_outputs)

            elif response_message.content:
                print(f"[模型生成结果]{response_message.content}")

                json_string_match = re.search(r"```json\s*(\{.*\})\s*```", response_message.content, re.DOTALL)

                if json_string_match:
                    extracted_json_content = json_string_match.group(1)
                    try:
                        final_response = json.loads(extracted_json_content)
                        print("Agent: 任务完成，成功解析最终JSON文案。")
                        return json.dumps(final_response, ensure_ascii=False, indent=2)
                    except json.JSONDecodeError as e:
                        print(f"Agent: 提取到JSON块但解析失败: {e}")
                        print(f"尝试解析的字符串:\n{extracted_json_content}")
                        messages.append(response_message) # 解析失败，继续对话
                else:
                    # 如果没有匹配到 ```json 块，尝试直接解析整个 content
                    try:
                        final_response = json.loads(response_message.content)
                        print("Agent: 任务完成，直接解析最终JSON文案。")
                        return json.dumps(final_response, ensure_ascii=False, indent=2)
                    except json.JSONDecodeError:
                        print("Agent: 生成了非JSON格式内容或非Markdown JSON块，可能还在思考或出错。")
                        messages.append(response_message) # 非JSON格式，继续对话
                # --- END: 添加 JSON 提取和解析逻辑 ---
            else:
                print("Agent: 未知响应，可能需要更多交互。")
                break
                
        except Exception as e:
            print(f"调用 DeepSeek API 时发生错误: {e}")
            break

    print("\n⚠️ Agent 达到最大迭代次数或未能生成最终文案。请检查Prompt或增加迭代次数。")
    return "未能成功生成文案。"

In [25]:
import json

def format_rednote_for_markdown(json_string: str) -> str:
    """
    将 JSON 格式的小红书文案转换为 Markdown 格式，以便于阅读和发布。

    Args:
        json_string (str): 包含小红书文案的 JSON 字符串。
                           预计格式为 {"title": "...", "body": "...", "hashtags": [...], "emojis": [...]}

    Returns:
        str: 格式化后的 Markdown 文本。
    """
    try:
        data = json.loads(json_string)
    except json.JSONDecodeError as e:
        return f"错误：无法解析 JSON 字符串 - {e}\n原始字符串：\n{json_string}"

    title = data.get("title", "无标题")
    body = data.get("body", "")
    hashtags = data.get("hashtags", [])
    # 表情符号通常已经融入标题和正文中，这里可以选择是否单独列出
    # emojis = data.get("emojis", []) 

    # 构建 Markdown 文本
    markdown_output = f"## {title}\n\n" # 标题使用二级标题
    
    # 正文，保留换行符
    markdown_output += f"{body}\n\n"
    
    # Hashtags
    if hashtags:
        hashtag_string = " ".join(hashtags) # 小红书标签通常是空格分隔
        markdown_output += f"{hashtag_string}\n"
        
    # 如果需要，可以单独列出表情符号，但通常它们已经包含在标题和正文中
    # if emojis:
    #     emoji_string = " ".join(emojis)
    #     markdown_output += f"\n使用的表情：{emoji_string}\n"
        
    return markdown_output.strip() # 去除末尾多余的空白

In [26]:
question_1 = "白羊座和狮子座配不配？"
tone_style_1 = "搞怪幽默"
result_1 = generate_rednote(question_1,tone_style_1)
print("\n--- 生成的文案 1  

      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      ---")
print(result_1)

markdown_note = format_rednote_for_markdown(result_1)
print("\n--- 格式化后文案（Markdown） 1---")
print(markdown_note)


🚀 启动小红书文案生成助手，产品：白羊座和狮子座配不配？，风格：搞怪幽默

-- Iteration 1 --
Agent: 决定调用工具....
Agent Action: 调用工具 'query_product_database', 参数：{'product_name': '白羊座'}


Creating embeddings: 100%|██████████████████████████████████████████████████████████| 14/14 [00:00<00:00, 129912.07it/s]


[
    [
        "\u2653 \u53cc\u9c7c\u5ea7 (2.19-3.20)\n**\u7279\u8d28**\uff1a\u60f3\u8c61\u529b\u4e30\u5bcc\u3001\u5171\u60c5\u529b\u5f3a\u3001\u827a\u672f\u5929\u8d4b  \n**\u5b88\u62a4\u661f**\uff1a\u6d77\u738b\u661f  \n**\u7537\u6027\u7279\u8d28**\uff1a\u6d6a\u6f2b\u4e3b\u4e49\u8005\uff0c\u6613\u53d7\u73af\u5883\u5f71\u54cd  \n**\u5973\u6027\u7279\u8d28**\uff1a\u6e29\u67d4\u6cbb\u6108\u7cfb\uff0c\u76f4\u89c9\u654f\u9510  \n**\u611f\u60c5\u8868\u73b0**\uff1a\u4e3a\u7231\u727a\u7272\u578b\uff0c\u5bb9\u6613\u9677\u5165\u5e7b\u60f3  \n**\u4e8b\u4e1a\u6807\u7b7e**\uff1a\u827a\u672f\u521b\u4f5c\uff5c\u6148\u5584\u4e8b\u4e1a\uff5c\u7597\u6108\u9886\u57df",
        0.6365807056427002
    ],
    [
        "\u264d \u5904\u5973\u5ea7 (8.23-9.22)\n**\u7279\u8d28**\uff1a\u903b\u8f91\u4e25\u8c28\u3001\u6ce8\u91cd\u7ec6\u8282\u3001\u670d\u52a1\u7cbe\u795e  \n**\u5b88\u62a4\u661f**\uff1a\u6c34\u661f  \n**\u7537\u6027\u7279\u8d28**\uff1a\u7406\u6027\u5206\u6790\u80fd\u529b\u5f3a  \n**\u5973\u6027\u7

Creating embeddings: 100%|███████████████████████████████████████████████████████████| 14/14 [00:00<00:00, 76260.07it/s]


[
    [
        "\u2653 \u53cc\u9c7c\u5ea7 (2.19-3.20)\n**\u7279\u8d28**\uff1a\u60f3\u8c61\u529b\u4e30\u5bcc\u3001\u5171\u60c5\u529b\u5f3a\u3001\u827a\u672f\u5929\u8d4b  \n**\u5b88\u62a4\u661f**\uff1a\u6d77\u738b\u661f  \n**\u7537\u6027\u7279\u8d28**\uff1a\u6d6a\u6f2b\u4e3b\u4e49\u8005\uff0c\u6613\u53d7\u73af\u5883\u5f71\u54cd  \n**\u5973\u6027\u7279\u8d28**\uff1a\u6e29\u67d4\u6cbb\u6108\u7cfb\uff0c\u76f4\u89c9\u654f\u9510  \n**\u611f\u60c5\u8868\u73b0**\uff1a\u4e3a\u7231\u727a\u7272\u578b\uff0c\u5bb9\u6613\u9677\u5165\u5e7b\u60f3  \n**\u4e8b\u4e1a\u6807\u7b7e**\uff1a\u827a\u672f\u521b\u4f5c\uff5c\u6148\u5584\u4e8b\u4e1a\uff5c\u7597\u6108\u9886\u57df",
        0.6365807056427002
    ],
    [
        "\u264d \u5904\u5973\u5ea7 (8.23-9.22)\n**\u7279\u8d28**\uff1a\u903b\u8f91\u4e25\u8c28\u3001\u6ce8\u91cd\u7ec6\u8282\u3001\u670d\u52a1\u7cbe\u795e  \n**\u5b88\u62a4\u661f**\uff1a\u6c34\u661f  \n**\u7537\u6027\u7279\u8d28**\uff1a\u7406\u6027\u5206\u6790\u80fd\u529b\u5f3a  \n**\u5973\u6027\u7

In [27]:
question_2 = "摩羯座适合找什么工作？"
tone_style_2 = "活泼甜美"
result_2 = generate_rednote(question_2,tone_style_2)
print("\n--- 生成的文案 2  ---")
print(result_2)

markdown_note = format_rednote_for_markdown(result_2)
print("\n--- 格式化后文案（Markdown） 1---")
print(markdown_note)


🚀 启动小红书文案生成助手，产品：摩羯座适合找什么工作？，风格：活泼甜美

-- Iteration 1 --
Agent: 决定调用工具....
Agent Action: 调用工具 'query_product_database', 参数：{'product_name': '摩羯座'}


Creating embeddings: 100%|██████████████████████████████████████████████████████████| 14/14 [00:00<00:00, 114019.91it/s]


[
    [
        "\u2653 \u53cc\u9c7c\u5ea7 (2.19-3.20)\n**\u7279\u8d28**\uff1a\u60f3\u8c61\u529b\u4e30\u5bcc\u3001\u5171\u60c5\u529b\u5f3a\u3001\u827a\u672f\u5929\u8d4b  \n**\u5b88\u62a4\u661f**\uff1a\u6d77\u738b\u661f  \n**\u7537\u6027\u7279\u8d28**\uff1a\u6d6a\u6f2b\u4e3b\u4e49\u8005\uff0c\u6613\u53d7\u73af\u5883\u5f71\u54cd  \n**\u5973\u6027\u7279\u8d28**\uff1a\u6e29\u67d4\u6cbb\u6108\u7cfb\uff0c\u76f4\u89c9\u654f\u9510  \n**\u611f\u60c5\u8868\u73b0**\uff1a\u4e3a\u7231\u727a\u7272\u578b\uff0c\u5bb9\u6613\u9677\u5165\u5e7b\u60f3  \n**\u4e8b\u4e1a\u6807\u7b7e**\uff1a\u827a\u672f\u521b\u4f5c\uff5c\u6148\u5584\u4e8b\u4e1a\uff5c\u7597\u6108\u9886\u57df",
        0.6365807056427002
    ],
    [
        "\u264d \u5904\u5973\u5ea7 (8.23-9.22)\n**\u7279\u8d28**\uff1a\u903b\u8f91\u4e25\u8c28\u3001\u6ce8\u91cd\u7ec6\u8282\u3001\u670d\u52a1\u7cbe\u795e  \n**\u5b88\u62a4\u661f**\uff1a\u6c34\u661f  \n**\u7537\u6027\u7279\u8d28**\uff1a\u7406\u6027\u5206\u6790\u80fd\u529b\u5f3a  \n**\u5973\u6027\u7

In [29]:
question_3 = "狮子女和天蝎男配吗？"
tone_style_3 = "温柔典雅"
result_3 = generate_rednote(question_3,tone_style_3)
print("\n--- 生成的文案 3  ---")
print(result_3)

markdown_note = format_rednote_for_markdown(result_3)
print("\n--- 格式化后文案（Markdown） 3---")
print(markdown_note)


🚀 启动小红书文案生成助手，产品：狮子女和天蝎男配吗？，风格：温柔典雅

-- Iteration 1 --
Agent: 决定调用工具....
Agent Action: 调用工具 'query_product_database', 参数：{'product_name': '狮子座'}


Creating embeddings: 100%|███████████████████████████████████████████████████████████| 14/14 [00:00<00:00, 77878.32it/s]


[
    [
        "\u2653 \u53cc\u9c7c\u5ea7 (2.19-3.20)\n**\u7279\u8d28**\uff1a\u60f3\u8c61\u529b\u4e30\u5bcc\u3001\u5171\u60c5\u529b\u5f3a\u3001\u827a\u672f\u5929\u8d4b  \n**\u5b88\u62a4\u661f**\uff1a\u6d77\u738b\u661f  \n**\u7537\u6027\u7279\u8d28**\uff1a\u6d6a\u6f2b\u4e3b\u4e49\u8005\uff0c\u6613\u53d7\u73af\u5883\u5f71\u54cd  \n**\u5973\u6027\u7279\u8d28**\uff1a\u6e29\u67d4\u6cbb\u6108\u7cfb\uff0c\u76f4\u89c9\u654f\u9510  \n**\u611f\u60c5\u8868\u73b0**\uff1a\u4e3a\u7231\u727a\u7272\u578b\uff0c\u5bb9\u6613\u9677\u5165\u5e7b\u60f3  \n**\u4e8b\u4e1a\u6807\u7b7e**\uff1a\u827a\u672f\u521b\u4f5c\uff5c\u6148\u5584\u4e8b\u4e1a\uff5c\u7597\u6108\u9886\u57df",
        0.6365807056427002
    ],
    [
        "\u264d \u5904\u5973\u5ea7 (8.23-9.22)\n**\u7279\u8d28**\uff1a\u903b\u8f91\u4e25\u8c28\u3001\u6ce8\u91cd\u7ec6\u8282\u3001\u670d\u52a1\u7cbe\u795e  \n**\u5b88\u62a4\u661f**\uff1a\u6c34\u661f  \n**\u7537\u6027\u7279\u8d28**\uff1a\u7406\u6027\u5206\u6790\u80fd\u529b\u5f3a  \n**\u5973\u6027\u7

Creating embeddings: 100%|███████████████████████████████████████████████████████████| 14/14 [00:00<00:00, 55084.67it/s]


[
    [
        "\u2653 \u53cc\u9c7c\u5ea7 (2.19-3.20)\n**\u7279\u8d28**\uff1a\u60f3\u8c61\u529b\u4e30\u5bcc\u3001\u5171\u60c5\u529b\u5f3a\u3001\u827a\u672f\u5929\u8d4b  \n**\u5b88\u62a4\u661f**\uff1a\u6d77\u738b\u661f  \n**\u7537\u6027\u7279\u8d28**\uff1a\u6d6a\u6f2b\u4e3b\u4e49\u8005\uff0c\u6613\u53d7\u73af\u5883\u5f71\u54cd  \n**\u5973\u6027\u7279\u8d28**\uff1a\u6e29\u67d4\u6cbb\u6108\u7cfb\uff0c\u76f4\u89c9\u654f\u9510  \n**\u611f\u60c5\u8868\u73b0**\uff1a\u4e3a\u7231\u727a\u7272\u578b\uff0c\u5bb9\u6613\u9677\u5165\u5e7b\u60f3  \n**\u4e8b\u4e1a\u6807\u7b7e**\uff1a\u827a\u672f\u521b\u4f5c\uff5c\u6148\u5584\u4e8b\u4e1a\uff5c\u7597\u6108\u9886\u57df",
        0.6365807056427002
    ],
    [
        "\u264d \u5904\u5973\u5ea7 (8.23-9.22)\n**\u7279\u8d28**\uff1a\u903b\u8f91\u4e25\u8c28\u3001\u6ce8\u91cd\u7ec6\u8282\u3001\u670d\u52a1\u7cbe\u795e  \n**\u5b88\u62a4\u661f**\uff1a\u6c34\u661f  \n**\u7537\u6027\u7279\u8d28**\uff1a\u7406\u6027\u5206\u6790\u80fd\u529b\u5f3a  \n**\u5973\u6027\u7